In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1uvszLurLwWeWjqEjk7FuGL68VspWsRMN'}) 
downloaded.GetContentFile('Consumer_Complaints_test_share.csv')

In [0]:
downloaded = drive.CreateFile({'id':'1K-Vfw1VLc9EosJiyXRrQH3-c51esMUac'}) 
downloaded.GetContentFile('Consumer_Complaints_train.csv')

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.model_selection import train_test_split,KFold
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neural_network import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import StandardScaler
import datetime
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#datafile_train=r"C:\Users\MANOJ GARIKAPATI\Python_ML\Project_1\P1 Data\Consumer_Complaints_train.csv"
#datafile_test=r"C:\Users\MANOJ GARIKAPATI\Python_ML\Project_1\P1 Data\Consumer_Complaints_test_share.csv"
cd_train=pd.read_csv('Consumer_Complaints_train.csv')
cd_test=pd.read_csv('Consumer_Complaints_test_share.csv')

In [7]:
from nltk.corpus import stopwords
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [0]:
stopwords_list = stopwords.words('english')
xtra_words = ["XX","XXX","XXXX","XXXXX","XXXXXX","XXXXXXX","XXXXXXXX"]

In [9]:
for i in range(len(xtra_words)):
    print(xtra_words[i])
    stopwords_list.append(xtra_words[i])

XX
XXX
XXXX
XXXXX
XXXXXX
XXXXXXX
XXXXXXXX


In [0]:
cd_train.loc[pd.isnull(cd_train["Consumer complaint narrative"]),'Consumer complaint narrative']="missing"

In [0]:
cd_test.loc[pd.isnull(cd_test["Consumer complaint narrative"]),'Consumer complaint narrative']="missing"

In [0]:
cd_train_tc=cd_train[pd.notnull(cd_train['Consumer complaint narrative'])]

In [13]:
cd_train_tc[pd.notnull(cd_train_tc['Consumer complaint narrative'])].shape

(478421, 18)

In [0]:
labels = cd_train_tc['Consumer disputed?']

In [0]:
text = cd_train_tc['Consumer complaint narrative']

In [0]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0.001, max_df=0.2,norm='l2', encoding='latin-1', stop_words=stopwords_list)

In [17]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(text, labels):
    print("Train:", train_index, "Test:", test_index)
    x_train, x_test = text.iloc[train_index], text.iloc[test_index]
    y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

Train: [340756 284343  57339 ... 280676 221528 401206] Test: [218196 359053  25215 ... 427356 237225   2778]


In [0]:
train_features = tfidf.fit_transform(x_train)
test_features = tfidf.transform(x_test)

In [19]:
clf_MNB = MultinomialNB()
clf_MNB.fit(train_features,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
MNB_preds = clf_MNB.predict(test_features)
MNB_preds

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype='<U3')

In [21]:
from sklearn.metrics import roc_auc_score
predictions_lab=np.where(MNB_preds=="Yes",1,0)
y_test_lab=np.where(y_test=="Yes",1,0)
roc_auc_score(y_test_lab,predictions_lab)

0.5002882584398608

In [22]:
train_features.shape

(382736, 1574)

In [0]:
clf_BNB = BernoulliNB()
clf_BNB.fit(train_features,y_train)

In [0]:
BNB_preds = clf_MNB.predict(test_features)
BNB_preds

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype='<U3')

In [0]:
from sklearn.metrics import roc_auc_score
predictions_lab=np.where(BNB_preds=="Yes",1,0)
y_test_lab=np.where(y_test=="Yes",1,0)
roc_auc_score(y_test_lab,predictions_lab)

0.5003840899354407

In [0]:
train = cd_train[cd_train['Consumer complaint narrative'] != 'None or Not Provided']

In [0]:
X_word = train['Consumer complaint narrative'].values
y = train['Consumer disputed?'].values

In [0]:
X_word.shape

(478421,)

In [0]:
X_train_word, X_test_word, y_train, y_test = train_test_split(X_word, y, test_size=0.20, random_state=67)
vectorizer = TfidfVectorizer(stop_words='english',lowercase=False, min_df=0.001, max_df = 0.2,
                             )
words_matrix_train = vectorizer.fit_transform(X_train_word)
words_matrix_test = vectorizer.transform(X_test_word)
words_matrix = vectorizer.transform(X_word)

In [0]:
words_matrix.shape

(478421, 1574)

In [0]:
#MultinomialNB classifier directly using tfidf text

model = MultinomialNB()
model.fit(words_matrix_train, y_train)

##Model evaluation via 'Consumer complaint narrative' only

model.score(words_matrix_test,y_test)

0.7875842608559336

In [0]:
preds_NB = model.predict(words_matrix_test)
preds_NB

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype='<U3')

In [0]:
from sklearn.metrics import roc_auc_score
predictions_lab=np.where(preds_NB=="Yes",1,0)
y_test_lab=np.where(y_test=="Yes",1,0)
roc_auc_score(y_test_lab,predictions_lab)

0.4999933652685076

In [0]:
# random forest directly using tfidf
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight='balanced' )
rfc.fit(words_matrix_train, y_train)

NameError: ignored

In [0]:
rfc

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [0]:
rfc.score(words_matrix_test,y_test)

0.7881381616763338

In [0]:
preds_RF = rfc.predict(words_matrix_test)
from sklearn.metrics import roc_auc_score
predictions_lab=np.where(preds_RF=="Yes",1,0)
y_test_lab=np.where(y_test=="Yes",1,0)
roc_auc_score(y_test_lab,predictions_lab)

0.5044773527677803